### Data

In [ ]:
# %cd SOICTHackathon2024

In [ ]:
import os 

In [ ]:
# train_folder = '/kaggle/input/grayscale-dataset1/grayscale_train/grayscale_train/train'

# val_folder = '/kaggle/input/grayscale-dataset1/grayscale_train/grayscale_train/validation'

In [ ]:
# train_img_list = os.listdir(train_folder + '/images')

In [ ]:
# from PIL import Image

# from IPython.display import display



# im = Image.open(train_folder + '/images/' + train_img_list[25])

# width, height = im.size

# display(im)

In [ ]:
# print(width, height)

In [ ]:
# !pip install -U albumentations

In [ ]:
# import albumentations as A

# from albumentations.pytorch import ToTensorV2

# from torch.utils.data import Dataset, DataLoader

# import cv2

# import torch





# class CustomYOLODataset(Dataset):

#     def __init__(self, images_dir, labels_dir, img_size=640, mode='train'):

#         """

#         Custom dataset for YOLO.



#         Args:

#             images_dir (str): Directory with images.

#             labels_dir (str): Directory with label files.

#             img_size (int): Target image size (YOLO often expects square images).

#             transform (callable, optional): Optional transform to apply to images.

#         """



#         self.img_size = img_size

#         self.images_dir = images_dir

#         self.labels_dir = labels_dir

#         if mode == 'train':

#             self.transform = A.Compose([

#                 A.RandomCrop(width=640, height=640),

#                 A.HorizontalFlip(p=0.5),

#                 A.VerticalFlip(p=0.5),

#                 A.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1),

#                 A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),

#                 ToTensorV2()  # Convert to PyTorch tensor

#             ])



#         else:

#             self.transform = A.Compose([

#                 A.Resize(width=640, height=640),

#                 A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),

#                 ToTensorV2()  # Convert to PyTorch tensor

#             ])

#         self.image_files = [f for f in os.listdir(images_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]

#         self.label_files = [f for f in os.listdir(labels_dir) if f.endswith('.txt')]

        



#     def __len__(self):

#         return len(self.image_files)



#     def __getitem__(self, idx):

#         # Load image

#         img_path = os.path.join(self.images_dir, self.image_files[idx])

#         image = cv2.imread(img_path)

#         image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        

#         # Resize image to target size

#         image = cv2.resize(image, (self.img_size, self.img_size))

        

#         # Load label

#         label_path = os.path.join(self.labels_dir, self.label_files[idx])



#         # Convert image to tensor and normalize

#         image = torch.from_numpy(image).float() / 255.0  # Normalize to [0,1]

#         image = image.permute(2, 0, 1)  # Change to (C, H, W) format for PyTorch



#         # Apply any data transformations

#         if self.transform:

#             image = self.transform(image)

        

#         boxes = []

#         with open(label_path, 'r') as f:

#             for line in f.readlines():

#                 label = line.strip().split()

#                 # YOLO labels: [class, x_center, y_center, width, height]

#                 class_id = int(label[0])

#                 x_center, y_center, width, height = map(float, label[1:])

#                 boxes.append([class_id, x_center, y_center, width, height])



#         return image, boxes


### Model

In [ ]:
%pip install -U ultralytics

In [ ]:
from ultralytics import YOLO, settings, RTDETR

In [ ]:
print(settings)

In [ ]:
# Update multiple settings

settings.update({"datasets_dir": "/kaggle/input/grayscale-dataset1", 

                 "tensorboard": False, 

                 "wandb": True})


In [ ]:
import os 
os.environ['WANDB_API_KEY'] = '0914f403b30196f64eb9744f9310b0029205c241'

In [ ]:
# !pip install torch_xla

In [ ]:
import torch
# import torch_xla.core.xla_model as xm

device = 0

In [ ]:
# check config
from ultralytics.cfg import get_cfg
overrides = {
        'model': '/kaggle/working/yolo11n.pt',
        'data': "/kaggle/input/grayscale-dataset1/grayscale_traindata copy.yaml",
        'imgsz': 640,
        'epochs': 101,
        'batch': 64,
        'optimizer': 'adam',
        'device': device,
        'save_period': 5,
        'augment': True,
        'workers': 8,
        'dropout': 0.1,
        'seed': 42,
        'label_smoothing': 0.0,
        'nbs': 64,
        'hsv_h': 0.015,
        'hsv_s': 0.7,
        'hsv_v': 0.4,
        'degrees': 0.0,
        'translate': 0.1,
        'scale': 0.5,
        'shear': 0.0,
        'perspective': 0.0,
        'flipud': 0.0,
        'fliplr': 0.5,
        'bgr': 0.0,
        'mosaic': 1.0,
        'mixup': 0.5,
        'copy_paste': 0.5,
        'copy_paste_mode': 'mixup'
    }
config = get_cfg(overrides=overrides)

config

In [ ]:
# using trainer
from ultralytics.models.yolo.detect import DetectionTrainer
from ultralytics import RTDETR, YOLO
from ultralytics.nn.tasks import RTDETRDetectionModel, DetectionModel


class CustomTrainer(DetectionTrainer):
    def get_model(self, cfg=None, weights=None, verbose=True):
        """Return a YOLO detection model."""
        model = DetectionModel(cfg, nc=self.data["nc"], verbose=verbose)
        if weights:
            model.load(weights)
        return model
    
trainer = CustomTrainer(
    overrides=overrides
)

trainer.start_epoch = 1

trainer.train()